In [1]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

A dataset containing details about Metro Nashville Police Department reported incidents is available at https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs. Make use of the API to find all aggravated burglary incidents () that were reported during the nine month period from January 1, 2022 through September 30, 2021. (Hint: Check out the API Docs to see how to narrow down the response to just the desired results).

In [2]:
#Bringing the API from TN police filtered by date and offense 

incident_reported = "https://data.nashville.gov/resource/2u6v-ujjs.json?$where=incident_reported between '2022-01-01T00:00:00' and '2022-09-30T23:59:59'&offense_description=BURGLARY- AGGRAVATED"
response = requests.get(incident_reported)

In [3]:
response

<Response [200]>

In [4]:
burglary = response.json()
burglary

[{'primary_key': '20220167824_11',
  'incident_number': '20220167824',
  'report_type': 'D',
  'report_type_description': 'DISPATCHED',
  'incident_status_code': 'O',
  'incident_status_description': 'OPEN',
  'investigation_status': 'Open',
  'incident_occurred': '2022-04-07T15:00:00.000',
  'incident_reported': '2022-04-08T11:52:00.000',
  'incident_location': 'JACKSON ST',
  'latitude': '36.17',
  'longitude': '-86.8',
  'location_code': '22',
  'location_description': 'RESIDENCE, HOME',
  'offense_number': '1',
  'offense_nibrs': '220',
  'offense_description': 'BURGLARY- AGGRAVATED',
  'weapon_primary': '16',
  'weapon_description': 'Unarmed',
  'victim_number': '1',
  'domestic_related': False,
  'victim_type': 'I',
  'victim_description': 'INDIVIDUAL (18 AND OVER)',
  'victim_gender': 'U',
  'victim_race': 'W',
  'victim_ethnicity': 'Non-Hispanic',
  'victim_county_resident': 'NON RESIDENT',
  'mapped_location': {'type': 'Point', 'coordinates': [-86.8, 36.17]}},
 {'primary_key':

In [5]:
#converting the data into a df
burglary_df = pd.DataFrame(response.json())
burglary_df

,primary_key,incident_number,report_type,report_type_description,incident_status_code,incident_status_description,investigation_status,incident_occurred,incident_reported,incident_location,...,victim_type,victim_description,victim_gender,victim_race,victim_ethnicity,victim_county_resident,mapped_location,rpa,zone,zip_code
0,20220167824_11,20220167824,D,DISPATCHED,O,OPEN,Open,2022-04-07T15:00:00.000,2022-04-08T11:52:00.000,JACKSON ST,...,I,INDIVIDUAL (18 AND OVER),U,W,Non-Hispanic,NON RESIDENT,"{'type': 'Point', 'coordinates': [-86.8, 36.17]}",NaN,NaN,NaN
1,20220126184_31,20220126184,D,DISPATCHED,O,OPEN,Open,2022-03-18T02:30:00.000,2022-03-18T06:51:00.000,BENTON AVE,...,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.77, 36.13]}",8029,817,NaN
2,20220027854_12,20220027854,D,DISPATCHED,O,OPEN,Open,2022-01-18T07:45:00.000,2022-01-19T23:48:00.000,CANE RIDGE RD,...,I,INDIVIDUAL (18 AND OVER),M,B,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.66, 36.04]}",NaN,NaN,NaN
3,20220032825_11,20220032825,D,DISPATCHED,O,OPEN,Open,2022-01-23T00:40:00.000,2022-01-24T06:57:00.000,BROOKWOOD TER,...,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.86, 36.13]}",5019,121,NaN
4,20220010799_11,20220010799,D,DISPATCHED,O,OPEN,Open,2021-12-06T12:00:00.000,2022-01-21T16:25:00.000,SOUTH ST,...,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.78, 36.15]}",6805,815,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,20220386358_13,20220386358,D,DISPATCHED,O,OPEN,Open,2022-08-06T00:00:00.000,2022-08-06T03:26:00.000,E TRINITY LN,...,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,NON RESIDENT,"{'type': 'Point', 'coordinates': [-86.74, 36.2]}",NaN,NaN,NaN
996,20220376574_41,20220376574,D,DISPATCHED,A,CLEARED BY ARREST,Closed,2022-07-31T20:55:00.000,2022-07-31T23:22:00.000,1017 1017,...,I,INDIVIDUAL (18 AND OVER),M,B,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.667, 36....",NaN,NaN,37013
997,20220389939_11,20220389939,D,DISPATCHED,O,OPEN,Open,2022-08-08T08:45:00.000,2022-08-08T12:28:00.000,NaN,...,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.65, 36.09]}",NaN,NaN,NaN
998,20220386358_14,20220386358,D,DISPATCHED,O,OPEN,Open,2022-08-06T00:00:00.000,2022-08-06T03:26:00.000,E TRINITY LN,...,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,NON RESIDENT,"{'type': 'Point', 'coordinates': [-86.74, 36.2]}",NaN,NaN,NaN


Using the 2020 American Community Survey API, obtain, for each census tract, the population (B01001_001E in the detailed tables) and the median income (S1901_C01_012E in the subject tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37.

In [6]:
endpoint = 'https://api.census.gov/data/2020/acs/acs5/variables.html'

response = requests.get(endpoint)

In [7]:
census = f'https://api.census.gov/data/2020/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:47'

response = requests.get(census)

response

<Response [200]>

In [8]:
tn_census = pd.DataFrame(response.json())
tn_census = tn_census.drop(labels=0, axis=0)
tn_census = tn_census.rename(columns={0:'Name', 1:'Census', 2:'State_Code',
                                     3:'County_Code', 4: 'Tract'})
tn_census

,Name,Census,State_Code,County_Code,Tract
1,"Census Tract 190.04, Davidson County, Tennessee",4470,47,037,019004
2,"Census Tract 190.07, Davidson County, Tennessee",2904,47,037,019007
3,"Census Tract 190.08, Davidson County, Tennessee",6005,47,037,019008
4,"Census Tract 191.05, Davidson County, Tennessee",5707,47,037,019105
5,"Census Tract 191.06, Davidson County, Tennessee",4336,47,037,019106
...,...,...,...,...,...
1697,"Census Tract 801.03, Robertson County, Tennessee",6580,47,147,080103
1698,"Census Tract 801.04, Robertson County, Tennessee",7936,47,147,080104
1699,"Census Tract 802, Robertson County, Tennessee",5716,47,147,080200
1700,"Census Tract 803.01, Robertson County, Tennessee",2722,47,147,080301


In [9]:
tn_davidson_census = tn_census[tn_census['County_Code'] == '037']
tn_davidson_census

,Name,Census,State_Code,County_Code,Tract
1,"Census Tract 190.04, Davidson County, Tennessee",4470,47,037,019004
2,"Census Tract 190.07, Davidson County, Tennessee",2904,47,037,019007
3,"Census Tract 190.08, Davidson County, Tennessee",6005,47,037,019008
4,"Census Tract 191.05, Davidson County, Tennessee",5707,47,037,019105
5,"Census Tract 191.06, Davidson County, Tennessee",4336,47,037,019106
...,...,...,...,...,...
1651,"Census Tract 189.01, Davidson County, Tennessee",3280,47,037,018901
1652,"Census Tract 189.02, Davidson County, Tennessee",2704,47,037,018902
1653,"Census Tract 189.04, Davidson County, Tennessee",3410,47,037,018904
1654,"Census Tract 189.05, Davidson County, Tennessee",3292,47,037,018905


2.Using the 2020 American Community Survey API, obtain, for each census tract, the population (B01001_001E in the detailed tables) and the median income (S1901_C01_012E in the subject tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37.

In [10]:
# Question2: Median Income

median_income = f'https://api.census.gov/data/2020/acs/acs5/subject?get=NAME,S1901_C01_012E&for=tract:*&in=state:47'

response = requests.get(median_income)

response

<Response [200]>

In [11]:
tn_median_income = pd.DataFrame(response.json())
tn_davidson_median_income_df = tn_median_income.rename(columns ={0:'Name', 1: 'Median_Income', 2: 'State_Code', 3: 'County_Code', 4: 'Tract'})
tn_davidson_median_income_df = tn_davidson_median_income_df.drop(labels=0, axis=0)
tn_davidson_median_income_df

,Name,Median_Income,State_Code,County_Code,Tract
1,"Census Tract 190.04, Davidson County, Tennessee",44001,47,037,019004
2,"Census Tract 190.07, Davidson County, Tennessee",48485,47,037,019007
3,"Census Tract 190.08, Davidson County, Tennessee",50512,47,037,019008
4,"Census Tract 191.05, Davidson County, Tennessee",48271,47,037,019105
5,"Census Tract 191.06, Davidson County, Tennessee",65323,47,037,019106
...,...,...,...,...,...
1697,"Census Tract 801.03, Robertson County, Tennessee",78731,47,147,080103
1698,"Census Tract 801.04, Robertson County, Tennessee",65921,47,147,080104
1699,"Census Tract 802, Robertson County, Tennessee",63967,47,147,080200
1700,"Census Tract 803.01, Robertson County, Tennessee",50169,47,147,080301


In [12]:
burglary_census = burglary_df['incident_occurred']
burglary_census

0      2022-04-07T15:00:00.000
1      2022-03-18T02:30:00.000
2      2022-01-18T07:45:00.000
3      2022-01-23T00:40:00.000
4      2021-12-06T12:00:00.000
                ...           
995    2022-08-06T00:00:00.000
996    2022-07-31T20:55:00.000
997    2022-08-08T08:45:00.000
998    2022-08-06T00:00:00.000
999    2022-08-08T13:00:00.000
Name: incident_occurred, Length: 1000, dtype: object

3. Download the 2020 census tract shapefiles for Tennessee from https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html. (The FIPS code for Tennessee is 47). Perform a spatial join to determine the census tract in which each burglary incident occurred. 

In [13]:
tn_davidson_median_income_df['incident_occurred'] = burglary_census
tn_davidson_median_income_df

,Name,Median_Income,State_Code,County_Code,Tract,incident_occurred
1,"Census Tract 190.04, Davidson County, Tennessee",44001,47,037,019004,2022-03-18T02:30:00.000
2,"Census Tract 190.07, Davidson County, Tennessee",48485,47,037,019007,2022-01-18T07:45:00.000
3,"Census Tract 190.08, Davidson County, Tennessee",50512,47,037,019008,2022-01-23T00:40:00.000
4,"Census Tract 191.05, Davidson County, Tennessee",48271,47,037,019105,2021-12-06T12:00:00.000
5,"Census Tract 191.06, Davidson County, Tennessee",65323,47,037,019106,2022-01-18T07:45:00.000
...,...,...,...,...,...,...
1697,"Census Tract 801.03, Robertson County, Tennessee",78731,47,147,080103,NaN
1698,"Census Tract 801.04, Robertson County, Tennessee",65921,47,147,080104,NaN
1699,"Census Tract 802, Robertson County, Tennessee",63967,47,147,080200,NaN
1700,"Census Tract 803.01, Robertson County, Tennessee",50169,47,147,080301,NaN


In [14]:
#converting the dataframe to a geodataframe

tn_davidson_median_income_df = gpd.GeoDataFrame(tn_davidson_median_income_df)
tn_davidson_median_income_df

,Name,Median_Income,State_Code,County_Code,Tract,incident_occurred
1,"Census Tract 190.04, Davidson County, Tennessee",44001,47,037,019004,2022-03-18T02:30:00.000
2,"Census Tract 190.07, Davidson County, Tennessee",48485,47,037,019007,2022-01-18T07:45:00.000
3,"Census Tract 190.08, Davidson County, Tennessee",50512,47,037,019008,2022-01-23T00:40:00.000
4,"Census Tract 191.05, Davidson County, Tennessee",48271,47,037,019105,2021-12-06T12:00:00.000
5,"Census Tract 191.06, Davidson County, Tennessee",65323,47,037,019106,2022-01-18T07:45:00.000
...,...,...,...,...,...,...
1697,"Census Tract 801.03, Robertson County, Tennessee",78731,47,147,080103,NaN
1698,"Census Tract 801.04, Robertson County, Tennessee",65921,47,147,080104,NaN
1699,"Census Tract 802, Robertson County, Tennessee",63967,47,147,080200,NaN
1700,"Census Tract 803.01, Robertson County, Tennessee",50169,47,147,080301,NaN


In [15]:
income_tn = f'https://api.census.gov/data/2020/acs/acs5/subject?get=NAME,S0101_C01_001E&for=tract:*&in=state:47'

income_tn = requests.get(income_tn)

In [16]:
income_tn = pd.DataFrame((response.json()))
income_df = income_tn.rename(columns ={0:'name', 1: 'median_income', 2: 'state_Code', 3: 'county_code', 4: 'tract'})
income_df = income_df.drop(labels=0, axis=0)
income_df = gpd.GeoDataFrame(income_df)
income_df

,name,median_income,state_Code,county_code,tract
1,"Census Tract 190.04, Davidson County, Tennessee",44001,47,037,019004
2,"Census Tract 190.07, Davidson County, Tennessee",48485,47,037,019007
3,"Census Tract 190.08, Davidson County, Tennessee",50512,47,037,019008
4,"Census Tract 191.05, Davidson County, Tennessee",48271,47,037,019105
5,"Census Tract 191.06, Davidson County, Tennessee",65323,47,037,019106
...,...,...,...,...,...
1697,"Census Tract 801.03, Robertson County, Tennessee",78731,47,147,080103
1698,"Census Tract 801.04, Robertson County, Tennessee",65921,47,147,080104
1699,"Census Tract 802, Robertson County, Tennessee",63967,47,147,080200
1700,"Census Tract 803.01, Robertson County, Tennessee",50169,47,147,080301


In [18]:
#Bringing the file from census 2020

census_tract = gpd.read_file('./tl_2020_47_tract/')
census_tract = gpd.GeoDataFrame(census_tract)
census_tract

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,47,175,925200,47175925200,9252,Census Tract 9252,G5020,S,227429512,1667739,+35.7438100,-085.4940227,"POLYGON ((-85.61516 35.76106, -85.61509 35.761..."
1,47,175,925000,47175925000,9250,Census Tract 9250,G5020,S,480712883,1225717,+35.6695378,-085.4220628,"POLYGON ((-85.60513 35.70854, -85.60511 35.708..."
2,47,003,950201,47003950201,9502.01,Census Tract 9502.01,G5020,S,121774227,0,+35.6517480,-086.5575518,"POLYGON ((-86.64406 35.64029, -86.64375 35.642..."
3,47,003,950202,47003950202,9502.02,Census Tract 9502.02,G5020,S,110617191,700793,+35.5845755,-086.5790796,"POLYGON ((-86.66377 35.58189, -86.66367 35.582..."
4,47,093,003300,47093003300,33,Census Tract 33,G5020,S,5860088,229299,+36.0020586,-083.8371218,"POLYGON ((-83.86208 35.99255, -83.86207 35.992..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1696,47,073,050501,47073050501,505.01,Census Tract 505.01,G5020,S,83355996,0,+36.5659159,-082.7715246,"POLYGON ((-82.85011 36.54107, -82.84955 36.541..."
1697,47,073,050601,47073050601,506.01,Census Tract 506.01,G5020,S,23847669,177932,+36.5836443,-082.6318663,"POLYGON ((-82.68939 36.58793, -82.68936 36.588..."
1698,47,073,050602,47073050602,506.02,Census Tract 506.02,G5020,S,23386546,668716,+36.5467218,-082.6458382,"POLYGON ((-82.69446 36.54856, -82.69445 36.548..."
1699,47,073,050502,47073050502,505.02,Census Tract 505.02,G5020,S,32873931,311980,+36.5332207,-082.7586081,"POLYGON ((-82.82242 36.52006, -82.82171 36.520..."


In [19]:
selected_cols = ['NAMELSAD', 'geometry']

# Select those columns
census_track = census_tract[selected_cols]
census_track = census_track.rename(columns ={'NAMELSAD':'census', 'geometry':'geometry'})
census_track

,census,geometry
0,Census Tract 9252,"POLYGON ((-85.61516 35.76106, -85.61509 35.761..."
1,Census Tract 9250,"POLYGON ((-85.60513 35.70854, -85.60511 35.708..."
2,Census Tract 9502.01,"POLYGON ((-86.64406 35.64029, -86.64375 35.642..."
3,Census Tract 9502.02,"POLYGON ((-86.66377 35.58189, -86.66367 35.582..."
4,Census Tract 33,"POLYGON ((-83.86208 35.99255, -83.86207 35.992..."
...,...,...
1696,Census Tract 505.01,"POLYGON ((-82.85011 36.54107, -82.84955 36.541..."
1697,Census Tract 506.01,"POLYGON ((-82.68939 36.58793, -82.68936 36.588..."
1698,Census Tract 506.02,"POLYGON ((-82.69446 36.54856, -82.69445 36.548..."
1699,Census Tract 505.02,"POLYGON ((-82.82242 36.52006, -82.82171 36.520..."


In [20]:
cols_income = ['median_income', 'county_code', 'tract']

# Select those columns
income = income_df[cols_income]
income = income_df.rename(columns ={'median_income':'income', 'county_code':'county', 'tract':'tract'})

In [21]:
join = gpd.sjoin(income, census_track['geometry'])

ValueError: 'left_df' should be GeoDataFrame, got <class 'pandas.core.frame.DataFrame'>